In [3]:
import folium
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import colors

# -----------------------------
# 1. CSV 불러오기
# -----------------------------
hangang_df = pd.read_csv("../../../measure/SAF/result/safety_hangang_total_score.csv")
nonhangang_df = pd.read_csv("../../../measure/SAF/result/safety_nonhangang_total_score.csv")

# -----------------------------
# 2. GeoDataFrame 불러오기
# -----------------------------
gu = gpd.read_file("../../../map/result/gu.gpkg", layer="gu").to_crs(4326)
parks = gpd.read_file("../../../map/result/parks.gpkg", layer="hangang_parks").to_crs(4326)

# -----------------------------
# 3. merge
# -----------------------------
# 자치구별 MED_score
gu = gu.merge(
    nonhangang_df[["자치구", "MED_score"]],
    left_on="SIGUNGU_NM",
    right_on="자치구",
    how="left"
).drop(columns=["자치구"])

# 한강공원: park_name 영문 → 한글 변환
park_name_map = {
    "Banpo": "반포한강공원", "Gangseo": "강서한강공원", "Gwangnaru": "광나루한강공원",
    "Ichon": "이촌한강공원", "Jamsil": "잠실한강공원", "Jamwon": "잠원한강공원",
    "Mangwon": "망원한강공원", "Nanji": "난지한강공원", "Ttukseom": "뚝섬한강공원",
    "Yanghwa": "양화한강공원", "Yeouido": "여의도한강공원"
}
parks["park_name"] = parks["park_name"].map(park_name_map)

# 한강공원 MED_score merge
parks = parks.merge(
    hangang_df[["한강공원", "MED_score"]],
    left_on="park_name",
    right_on="한강공원",
    how="left"
).drop(columns=["한강공원"])

# -----------------------------
# 4. colormap (구 + 공원 전체 기준)
# -----------------------------
all_min = min(gu["MED_score"].min(), parks["MED_score"].min())
all_max = max(gu["MED_score"].max(), parks["MED_score"].max())

cmap = plt.cm.Oranges
norm = colors.Normalize(vmin=all_min, vmax=all_max)

# -----------------------------
# 5. folium 지도 생성
# -----------------------------
m = folium.Map(location=[37.5665,126.9780], zoom_start=11, tiles="CartoDB positron")

# 자치구 레이어
folium.GeoJson(
    gu,
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["MED_score"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["MED_score"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 0.6
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["SIGUNGU_NM", "MED_score"],
        aliases=["자치구", "의료 점수"]
    )
).add_to(m)

# 한강공원 레이어
folium.GeoJson(
    parks,
    name="한강공원",
    style_function=lambda f: {
        "fillColor": "#eeeeee" if pd.isna(f["properties"]["MED_score"]) else 
                     colors.rgb2hex(cmap(norm(f["properties"]["MED_score"]))),
        "color": "#666",
        "weight": 1.2,
        "fillOpacity": 0.8
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["park_name", "MED_score"],
        aliases=["공원", "의료 점수"]
    )
).add_to(m)

In [4]:
# 저장
folium.LayerControl(collapsed=False).add_to(m)
m.save("../../result/SAF/med_score_visualization.html")
print("med_score_visualization.html 저장 완료")

med_score_visualization.html 저장 완료
